# House Prices Attempt I: Linear Regression

For my first non-physics machine learning project I will be testing out various techniques on a straightforward house prices dataset. The first attempt will be a simple linear regression. The plan is to use one hot encoding (etc.) to deal with the categorical data which should be easy to do with the **pandas** toolset. I also expect some ordinal features will not lend well to a simple linear model, so I will also try making up some nonlinear features (*e.g.* (Overall Quality)^2).  I expect that the number of features may become unwieldly, but we will see :).

Standard imports:

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import numpy as np
import pandas as pd

Load raw training data:

In [3]:
data = pd.read_csv("../data/train.csv")
print(data.shape)

(1460, 81)


OK, so there are $m = 1460$ training examples and $n = 80$ features (ignoring 'Id'). So I'm pretty sure after one hot encoding $n$ will be approaching $m$. This may be bad but I'm going to allow it just to see what happens.

## One hot encoding
Here are the categorical features that I have decided to encode:

In [4]:
ohe_categories = [
    'MSZoning',
    'Street',
    'Alley',
    'LotShape',
    'LandContour',
    'LotConfig',
    'Neighborhood',
    'Condition1',
    'Condition2',
    'BldgType',
    'HouseStyle',
    'RoofStyle',
    'RoofMatl',
    'Exterior1st',
    'Exterior2nd',
    'MasVnrType',
    'Foundation',
    'Heating',
    'Electrical',
    'GarageType',
    'MiscFeature',
    'SaleType',
    'SaleCondition'
]
data_augmented_1 = pd.get_dummies(data[ohe_categories])
data_augmented_1.head()

,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,Alley_Grvl,Alley_Pave,LotShape_IR1,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,0,0,0,1,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0
1,0,0,0,1,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0
2,0,0,0,1,0,0,1,0,0,1,...,0,0,0,1,0,0,0,0,1,0
3,0,0,0,1,0,0,1,0,0,1,...,0,0,0,1,1,0,0,0,0,0
4,0,0,0,1,0,0,1,0,0,1,...,0,0,0,1,0,0,0,0,1,0


I will add the treated data to the *data_augmented* variable as I go.

## Ordinal data mapping

I will use the **pd.DataFrame.cat** functionality to map the ordinal data to numbers. I will just map them as natural numbers for now. Here are the categories that will need maping.

In [96]:
odm_ordered_lists = {
    'Utilities': ['ELO', 'NoSeWa', 'NoSewr', 'AllPub'],
    'LandSlope': ['Sev', 'Mod', 'Gtl'],
    'ExterQual': ['Po', 'Fa', 'TA', 'Gd', 'Ex'],
    'ExterCond': ['Po', 'Fa', 'TA', 'Gd', 'Ex'],
    'BsmtQual': ['Po', 'Fa', 'TA', 'Gd', 'Ex'],
    'BsmtCond': ['Po', 'Fa', 'TA', 'Gd', 'Ex'],
    'BsmtExposure': ['No', 'Mn', 'Av', 'Gd'],
    'BsmtFinType1': ['Unf', 'LwQ', 'Rec', 'BLQ', 'ALQ', 'GLQ'],
    'BsmtFinType2': ['Unf', 'LwQ', 'Rec', 'BLQ', 'ALQ', 'GLQ'],
    'HeatingQC': ['Po', 'Fa', 'TA', 'Gd', 'Ex'],
    'CentralAir': ['N','Y'],
    'KitchenQual': ['Po', 'Fa', 'TA', 'Gd', 'Ex'], 
    'Functional': ['Sal', 'Sev', 'Maj2', 'Maj1', 'Mod', 'Min2', 'Min1', 'Typ'],
    'FireplaceQu': ['Po', 'Fa', 'TA', 'Gd', 'Ex'],
    'GarageFinish': ['Unf', 'RFn','Fin'],
    'GarageQual': ['Po', 'Fa', 'TA', 'Gd', 'Ex'],
    'GarageCond': ['Po', 'Fa', 'TA', 'Gd', 'Ex'],
    'PavedDrive': ['Y', 'P', 'N'],
    'PoolQC': ['Fa', 'TA', 'Gd', 'Ex'],
    'Fence': ['MnWw', 'GdWo', 'MnPrv', 'GdPrv'],    
}

def index_exclude_nan(ordered_list,string):
    """The NaN's will be set to 0 since not having something is probably bad.
       Also add 1 to the indices so they are always natural numbers."""
    if isinstance(string, str):
        return ordered_list.index(string)+1
    else:
        return 0
        
def odm(series,ordered_list):
    "Quick and dirty way to map series entry to ordered_list index"
    return list(map(lambda x: index_exclude_nan(ordered_list,x),series))

data_augmented_2 = pd.DataFrame()
for cat in odm_ordered_lists:
    data_augmented_2[cat] = odm(data[cat],odm_ordered_lists[cat])

data_augmented_2.head()

,Utilities,LandSlope,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,HeatingQC,CentralAir,KitchenQual,Functional,FireplaceQu,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence
0,4,3,4,3,4,3,1,6,1,5,2,4,8,0,2,3,3,1,0,0
1,4,3,3,3,4,3,4,5,1,5,2,3,8,3,2,3,3,1,0,0
2,4,3,4,3,4,3,2,6,1,5,2,4,8,3,2,3,3,1,0,0
3,4,3,3,3,3,4,1,5,1,4,2,4,8,4,1,3,3,1,0,0
4,4,3,4,3,4,3,3,6,1,5,2,4,8,3,2,3,3,1,0,0


## Clean up remaining data

In [97]:
The numerical data can now be cleaned up by replacing NaN's with the average value of the column

SyntaxError: invalid syntax (<ipython-input-97-abc99c74c5c4>, line 1)

In [101]:
def is_used_category(cat):
    "Find all categorical and unwanted categories."
    return np.array([(c in ohe_categories) or (c in odm_ordered_lists) or (c in ['Id','SalePrice']) for c in cat])

def nan_to_avg(series):
    "Replace nan with series average."
    return series.fillna(np.mean(series))

data_augmented_3 = data.loc[:, np.invert(is_used_category(data.columns))]

data_augmented_3 = data_augmented_3.apply(nan_to_avg)

data_augmented_3.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,60,65.0,8450,7,5,2003,2003,196.0,706,0,...,548,0,61,0,0,0,0,0,2,2008
1,20,80.0,9600,6,8,1976,1976,0.0,978,0,...,460,298,0,0,0,0,0,0,5,2007
2,60,68.0,11250,7,5,2001,2002,162.0,486,0,...,608,0,42,0,0,0,0,0,9,2008
3,70,60.0,9550,7,5,1915,1970,0.0,216,0,...,642,0,35,272,0,0,0,0,2,2006
4,60,84.0,14260,8,5,2000,2000,350.0,655,0,...,836,192,84,0,0,0,0,0,12,2008


Now combine all augmented datasets into the final training dataset.

In [108]:
data_augmented = pd.concat([data_augmented_1,data_augmented_2,data_augmented_3],axis = 1)

data_augmented.head()

,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,Alley_Grvl,Alley_Pave,LotShape_IR1,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,0,0,0,1,0,0,1,0,0,0,...,548,0,61,0,0,0,0,0,2,2008
1,0,0,0,1,0,0,1,0,0,0,...,460,298,0,0,0,0,0,0,5,2007
2,0,0,0,1,0,0,1,0,0,1,...,608,0,42,0,0,0,0,0,9,2008
3,0,0,0,1,0,0,1,0,0,1,...,642,0,35,272,0,0,0,0,2,2006
4,0,0,0,1,0,0,1,0,0,1,...,836,192,84,0,0,0,0,0,12,2008


0        65.000000
1        80.000000
2        68.000000
3        60.000000
4        84.000000
5        85.000000
6        75.000000
7        70.049958
8        51.000000
9        50.000000
10       70.000000
11       85.000000
12       70.049958
13       91.000000
14       70.049958
15       51.000000
16       70.049958
17       72.000000
18       66.000000
19       70.000000
20      101.000000
21       57.000000
22       75.000000
23       44.000000
24       70.049958
25      110.000000
26       60.000000
27       98.000000
28       47.000000
29       60.000000
           ...    
1430     60.000000
1431     70.049958
1432     60.000000
1433     93.000000
1434     80.000000
1435     80.000000
1436     60.000000
1437     96.000000
1438     90.000000
1439     80.000000
1440     79.000000
1441     70.049958
1442     85.000000
1443     70.049958
1444     63.000000
1445     70.000000
1446     70.049958
1447     80.000000
1448     70.000000
1449     21.000000
1450     60.000000
1451     78.

In [88]:
data['LotFrontage']

0        65.0
1        80.0
2        68.0
3        60.0
4        84.0
5        85.0
6        75.0
7         NaN
8        51.0
9        50.0
10       70.0
11       85.0
12        NaN
13       91.0
14        NaN
15       51.0
16        NaN
17       72.0
18       66.0
19       70.0
20      101.0
21       57.0
22       75.0
23       44.0
24        NaN
25      110.0
26       60.0
27       98.0
28       47.0
29       60.0
        ...  
1430     60.0
1431      NaN
1432     60.0
1433     93.0
1434     80.0
1435     80.0
1436     60.0
1437     96.0
1438     90.0
1439     80.0
1440     79.0
1441      NaN
1442     85.0
1443      NaN
1444     63.0
1445     70.0
1446      NaN
1447     80.0
1448     70.0
1449     21.0
1450     60.0
1451     78.0
1452     35.0
1453     90.0
1454     62.0
1455     62.0
1456     85.0
1457     66.0
1458     68.0
1459     75.0
Name: LotFrontage, Length: 1460, dtype: float64